In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Input, Conv1D, MaxPooling1D, Flatten, Dropout



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/total-data/total_data3.csv


In [2]:
df = pd.read_csv("/kaggle/input/total-data/total_data3.csv")

In [3]:
df

,away_current_pos,away_last_yr_pos,away_prev_game_perf,away_team_av_points,away_team_av_points_conceded,away_team_away_form,away_team_form,away_team_rest_time,away_win_percentage,h2h_form,...,Away_Sao Judas,Away_Sesi,Away_Super Imperatriz,Away_UM Itapetininga,Away_Ulbra Sao Caetano,Away_Volei Futuro,Away_Volei Renata,Away_Volei Ribeirao,Away_Voleisul/Paqueta,Away_Volta Redonda
0,12,12,0,20.000000,20.000000,0.000000,0.000000,7,50.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,7,6,0,20.000000,20.000000,0.000000,0.000000,7,50.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,13,3,0,20.000000,20.000000,0.000000,0.000000,7,50.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,15,8,0,20.000000,20.000000,0.000000,0.000000,7,50.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,13,3,0,20.400000,20.640000,-0.200000,-0.200000,2,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,2,4,1,23.079388,21.498768,0.703523,1.005780,7,78.571429,0.075245,...,0,0,0,0,0,0,0,0,0,0
1286,1,3,1,24.507014,21.383092,1.925802,2.521766,4,92.857143,-0.660196,...,0,1,0,0,0,0,0,0,0,0
1287,2,4,1,23.290808,22.092545,-0.037181,0.763699,-362,76.666667,-0.071844,...,0,0,0,0,0,0,0,0,0,0
1288,1,3,0,23.791155,22.285179,0.940642,0.933930,4,86.666667,0.257475,...,0,1,0,0,0,0,0,0,0,0


In [4]:
y = df["result"]
X = df.drop(["result"], axis=1)

shape = int(0.8*X.shape[0])

X_train = X.iloc[0:shape, :]
X_test = X.iloc[shape:, :]
y_train = y.iloc[0:shape]
y_test = y.iloc[shape :]

In [5]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

scaled_X_train = scaled_X_train.reshape(scaled_X_train.shape[0],scaled_X_train.shape[1],1)
scaled_X_test = scaled_X_test.reshape(scaled_X_test.shape[0],scaled_X_test.shape[1],1)

In [6]:
print(scaled_X_train.shape)
scaled_X_test.shape

(1031, 84, 1)


(258, 84, 1)

In [7]:
lstm_out = 128
epochs = 30

In [8]:
model = Sequential()
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2, return_sequences=False, input_shape = (scaled_X_train.shape[1], 1))) #84 * 1
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 70,721
Trainable params: 70,721
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.fit(scaled_X_train, y_train, batch_size = 1, epochs = epochs)

Epoch 1/20
1031/1031 [==============================] - 120s 113ms/step - loss: 0.6921 - accuracy: 0.5567
Epoch 2/20
1031/1031 [==============================] - 116s 113ms/step - loss: 0.6866 - accuracy: 0.5436
Epoch 3/20
1031/1031 [==============================] - 117s 113ms/step - loss: 0.6866 - accuracy: 0.5555
Epoch 4/20
1031/1031 [==============================] - 115s 112ms/step - loss: 0.7035 - accuracy: 0.5463
Epoch 5/20
1031/1031 [==============================] - 116s 112ms/step - loss: 0.6911 - accuracy: 0.5409
Epoch 6/20
1031/1031 [==============================] - 115s 112ms/step - loss: 0.6835 - accuracy: 0.5877
Epoch 7/20
1031/1031 [==============================] - 115s 111ms/step - loss: 0.6885 - accuracy: 0.5624
Epoch 8/20
1031/1031 [==============================] - 116s 113ms/step - loss: 0.6891 - accuracy: 0.5525
Epoch 9/20
1031/1031 [==============================] - 119s 115ms/step - loss: 0.6926 - accuracy: 0.5349
Epoch 10/20
1031/1031 [=======================

In [11]:
y_pred = model.predict(scaled_X_test, verbose = 1)
y_pred

9/9 [==============================] - 0s 21ms/step


array([[0.55331963],
       [0.55331296],
       [0.5533118 ],
       [0.5534024 ],
       [0.5533197 ],
       [0.5533173 ],
       [0.55331916],
       [0.55331296],
       [0.5533184 ],
       [0.55331963],
       [0.5533152 ],
       [0.5533117 ],
       [0.55331933],
       [0.5533198 ],
       [0.55331916],
       [0.5534024 ],
       [0.55331933],
       [0.5533152 ],
       [0.5533183 ],
       [0.55331963],
       [0.5533152 ],
       [0.5533197 ],
       [0.55331284],
       [0.5533183 ],
       [0.5533197 ],
       [0.55331916],
       [0.55331516],
       [0.55331296],
       [0.55331516],
       [0.55331963],
       [0.5533198 ],
       [0.5534024 ],
       [0.55331933],
       [0.5533118 ],
       [0.55331963],
       [0.5533171 ],
       [0.5533197 ],
       [0.55331904],
       [0.5533152 ],
       [0.5533184 ],
       [0.55331296],
       [0.5533197 ],
       [0.5534024 ],
       [0.5533173 ],
       [0.55331516],
       [0.55331296],
       [0.55331963],
       [0.553

In [12]:
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0
y_pred = y_pred.astype('int32')
y_pred

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [13]:
accuracy_score(y_test, y_pred) #final result accuracy on test dataset

0.5232558139534884